In [28]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [29]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

### Question 1. What is the sum of the outputs of the generator for limit = 5?

In [30]:
limit = 5
generator = square_root_generator(limit)
print(sum(generator))

8.382332347441762


### What is the 13th number yielded by the generator?

In [31]:
limit = 13
generator = square_root_generator(limit)
for num in generator:
  print(num)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [32]:
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

append first generator

In [33]:
import dlt
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name="people")
info = generators_pipeline.run(people_1(), table_name="person", write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708274699.984986 is LOADED and contains no failed jobs


check loaded tables, and display preson table

In [34]:
import duckdb
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n person table below:")
persons = conn.sql("SELECT * FROM person").df()
display(persons)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person              │
│ person_merged       │
└─────────────────────┘




 person table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708274699.984986,ssxhYV7lpzTa2A,None
1,2,Person_2,27,City_A,1708274699.984986,HIUo/gRbwASdsA,None
2,3,Person_3,28,City_A,1708274699.984986,yOIVigqWLK5uxw,None
3,4,Person_4,29,City_A,1708274699.984986,T5rlOE6l5F+zOg,None
4,5,Person_5,30,City_A,1708274699.984986,5LlFuxbZ0ty3Lg,None


sum people ages

In [35]:
sum_ages = conn.sql("SELECT SUM(age) FROM person")
print(f" the sum of all ages of people {sum_ages}")

 the sum of all ages of people ┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘



append second generator

In [36]:
info = generators_pipeline.run(people_2(), table_name='person', write_disposition='append')

show person table after appending second generator

In [37]:
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n person table below:")
persons = conn.sql("SELECT * FROM person").df()
display(persons)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person              │
│ person_merged       │
└─────────────────────┘




 person table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708274699.984986,ssxhYV7lpzTa2A,None
1,2,Person_2,27,City_A,1708274699.984986,HIUo/gRbwASdsA,None
2,3,Person_3,28,City_A,1708274699.984986,yOIVigqWLK5uxw,None
3,4,Person_4,29,City_A,1708274699.984986,T5rlOE6l5F+zOg,None
4,5,Person_5,30,City_A,1708274699.984986,5LlFuxbZ0ty3Lg,None
5,3,Person_3,33,City_B,1708274700.627605,SRyqeJz1u8MhTw,Job_3
6,4,Person_4,34,City_B,1708274700.627605,wMQoJIVfmBql+g,Job_4
7,5,Person_5,35,City_B,1708274700.627605,JQ/hZ12rkbWPhg,Job_5
8,6,Person_6,36,City_B,1708274700.627605,rqPSUD9HGDVzPg,Job_6
9,7,Person_7,37,City_B,1708274700.627605,vKkkx9INsKvRSg,Job_7


sum of all ages of people

In [38]:
sum_ages = conn.sql("SELECT SUM(age) FROM person").df()
display(sum_ages)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


create new table

In [39]:
info = generators_pipeline.run(people_1(), table_name='person_merged', write_disposition='replace', primary_key='id')
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708274701.233263 is LOADED and contains no failed jobs


check table is created successfully

In [40]:
person_merged = conn.sql("SELECT * FROM person_merged").df()
display(person_merged)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708274701.233263,Ny2xXvmS4zNgaA,None
1,2,Person_2,27,City_A,1708274701.233263,lPskpqvX3GBUEA,None
2,3,Person_3,28,City_A,1708274701.233263,nI5/Qw/R4TlIVQ,None
3,4,Person_4,29,City_A,1708274701.233263,GJd1lJ7tz/m1QQ,None
4,5,Person_5,30,City_A,1708274701.233263,ot+Xt6V2pWwgpA,None


merge second generator

In [41]:
info = generators_pipeline.run(people_2(), table_name='person_merged', write_disposition='merge', primary_key='id')
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708274701.8622246 is LOADED and contains no failed jobs


In [42]:
person_merged = conn.sql("SELECT * FROM person_merged").df()
display(person_merged)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708274701.233263,Ny2xXvmS4zNgaA,None
1,2,Person_2,27,City_A,1708274701.233263,lPskpqvX3GBUEA,None
2,5,Person_5,35,City_B,1708274701.8622246,LrESKnXc2Hepeg,Job_5
3,4,Person_4,34,City_B,1708274701.8622246,JveDwQueJFpuWA,Job_4
4,8,Person_8,38,City_B,1708274701.8622246,iGZo5XlfRKN66g,Job_8
5,7,Person_7,37,City_B,1708274701.8622246,0GgKqi2EVoFHPw,Job_7
6,3,Person_3,33,City_B,1708274701.8622246,vWkjymjq8IxgAQ,Job_3
7,6,Person_6,36,City_B,1708274701.8622246,urBekmuPnD02FA,Job_6


check second generator is merged successfully

sum people ages

In [43]:
sum_ages = conn.sql("SELECT SUM(age) FROM person_merged")
print(f" the sum of all ages of people {sum_ages}")

 the sum of all ages of people ┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

